In [1]:
import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)

if not dir1 in sys.path:
    sys.path.append(dir1)

os.chdir('..')

In [2]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import yaml

from sklearn.metrics import mean_absolute_error as MAE
from matplotlib import pyplot as plt
from xgboost import XGBRegressor

# os.chdir('src/models/ts2vec_src')

from src.data.preprocessing import read_data, data_to_np_tensor, preprocess_split
from src.models.ts2vec_src.ts2vec import TS2Vec

%load_ext autoreload
%autoreload 2

# os.chdir('../../..')

# Preprocessing

In [3]:
# data = read_data('data/all_tickers.csv')
# train_data = preprocessing(
#     data, 
#     ['Open', 'High', 'Low', 'Close', 'Volume'],         
#     start_date = '2023-12-15',
#     end_date = '2023-12-21'
# )

# test_data = preprocessing(
#     data, 
#     ['Open', 'High', 'Low', 'Close', 'Volume'],          
#     start_date = '2023-12-21',
#     end_date = '2023-12-22',
#     tickers_save = train_data['Close'].columns
# )

In [4]:
# for feature in ['Open', 'High', 'Low', 'Close', 'Volume']:
#     print(train_data['Close'].shape[1], test_data['Close'].shape[1])

In [5]:
# train_ts = data_to_np_tensor(train_data)
# test_ts = data_to_np_tensor(test_data)

In [6]:
# model = TS2Vec(
#     input_dims=train_ts.shape[2],
#     device=1,
#     output_dims=320
# )

# loss_log = model.fit(
#     train_ts,
#     verbose=False
# )

# test_repr = model.encode(test_ts)
# #test_repr = model.encode(test_data, encoding_window='full_series')

In [7]:
# test_repr = model.encode(test_ts, encoding_window='full_series')
# print(test_repr.shape)
# test_repr

# mean stock price prediction

In [8]:
with open('configs/best_stocks_nans_rate.yaml') as f:
    best_stocks = yaml.load(f, Loader=yaml.FullLoader)
best_stocks = list(best_stocks.keys())
# best_stocks

In [9]:
df = read_data('data/all_tickers.csv')
df_best = df.query("Stock in @best_stocks")

In [10]:
df_best

,Date,Time,Open,High,Low,Close,Volume,Datetime,Stock,Day_week
61,2024-01-30,22:58:00,188.130,188.17,188.110,188.14,1500,2024-01-30 22:58:00,AAPL,Tuesday
62,2024-01-30,22:57:00,188.220,188.22,188.080,188.11,5700,2024-01-30 22:57:00,AAPL,Tuesday
63,2024-01-30,22:56:00,188.220,188.27,188.215,188.22,2600,2024-01-30 22:56:00,AAPL,Tuesday
64,2024-01-30,22:55:00,188.150,188.20,188.140,188.20,2100,2024-01-30 22:55:00,AAPL,Tuesday
65,2024-01-30,22:54:00,188.180,188.18,188.140,188.18,2013,2024-01-30 22:54:00,AAPL,Tuesday
...,...,...,...,...,...,...,...,...,...,...
95882,2023-01-30,17:34:00,115.050,115.09,114.960,114.97,800,2023-01-30 17:34:00,XOM,Monday
95883,2023-01-30,17:33:00,115.090,115.27,114.910,115.15,4603,2023-01-30 17:33:00,XOM,Monday
95884,2023-01-30,17:32:00,115.250,115.25,115.000,115.03,850,2023-01-30 17:32:00,XOM,Monday
95885,2023-01-30,17:31:00,115.145,115.16,115.145,115.16,237,2023-01-30 17:31:00,XOM,Monday


In [11]:
mask = df['Stock'].apply(lambda x: x in best_stocks) 

In [12]:
df[mask]

,Date,Time,Open,High,Low,Close,Volume,Datetime,Stock,Day_week
61,2024-01-30,22:58:00,188.130,188.17,188.110,188.14,1500,2024-01-30 22:58:00,AAPL,Tuesday
62,2024-01-30,22:57:00,188.220,188.22,188.080,188.11,5700,2024-01-30 22:57:00,AAPL,Tuesday
63,2024-01-30,22:56:00,188.220,188.27,188.215,188.22,2600,2024-01-30 22:56:00,AAPL,Tuesday
64,2024-01-30,22:55:00,188.150,188.20,188.140,188.20,2100,2024-01-30 22:55:00,AAPL,Tuesday
65,2024-01-30,22:54:00,188.180,188.18,188.140,188.18,2013,2024-01-30 22:54:00,AAPL,Tuesday
...,...,...,...,...,...,...,...,...,...,...
95882,2023-01-30,17:34:00,115.050,115.09,114.960,114.97,800,2023-01-30 17:34:00,XOM,Monday
95883,2023-01-30,17:33:00,115.090,115.27,114.910,115.15,4603,2023-01-30 17:33:00,XOM,Monday
95884,2023-01-30,17:32:00,115.250,115.25,115.000,115.03,850,2023-01-30 17:32:00,XOM,Monday
95885,2023-01-30,17:31:00,115.145,115.16,115.145,115.16,237,2023-01-30 17:31:00,XOM,Monday


In [13]:
dd = {'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'}

In [14]:
df_best_h = df_best.set_index('Datetime').groupby(
    ['Stock', pd.Grouper(freq='h')],
).agg(dd)

df_best_h

Open    High     Low   Close  Volume
Stock Datetime                                                   
AAPL  2023-01-30 17:00:00  145.15  145.53  144.18  145.13  291457
      2023-01-30 18:00:00  143.97  145.33  143.38  144.96  348830
      2023-01-30 19:00:00  144.07  144.30  143.50  143.95  215404
      2023-01-30 20:00:00  143.38  144.24  143.18  144.17  254970
      2023-01-30 21:00:00  143.31  143.81  143.01  143.38  192383
...                           ...     ...     ...     ...     ...
XOM   2024-01-30 18:00:00  103.69  103.70  102.95  103.22  159389
      2024-01-30 19:00:00  103.79  103.96  103.49  103.67   90110
      2024-01-30 20:00:00  103.68  103.98  103.63  103.76   54009
      2024-01-30 21:00:00  104.15  104.16  103.67  103.68   61363
      2024-01-30 22:00:00  104.38  104.46  104.15  104.15   91004

[42196 rows x 5 columns]

In [15]:
df = df_best_h.groupby('Stock').pct_change().reset_index()

In [16]:
df.head()

,Stock,Datetime,Open,High,Low,Close,Volume
0,AAPL,2023-01-30 17:00:00,NaN,NaN,NaN,NaN,NaN
1,AAPL,2023-01-30 18:00:00,-0.008130,-0.001374,-0.005549,-0.001171,0.196849
2,AAPL,2023-01-30 19:00:00,0.000695,-0.007087,0.000837,-0.006967,-0.382496
3,AAPL,2023-01-30 20:00:00,-0.004789,-0.000416,-0.002230,0.001528,0.183683
4,AAPL,2023-01-30 21:00:00,-0.000488,-0.002981,-0.001187,-0.005480,-0.245468


In [17]:
train_start, train_end = '2023-10-01', '2023-11-01'
test_start, test_end = '2023-11-01', '2023-11-07'


train_data = preprocess_split(
    df, 
    ['Open', 'High', 'Low', 'Close', 'Volume'],         
    start_date = train_start,
    end_date = train_end,
    tickers_save = best_stocks
)

test_data = preprocess_split(
    df, 
    ['Open', 'High', 'Low', 'Close', 'Volume'],          
    start_date = test_start,
    end_date = test_end,
    tickers_save = best_stocks
)

NAN in data  0
NAN in data  0
NAN in data  0
NAN in data  0
NAN in data  0
NAN in data  0
NAN in data  0
NAN in data  0
NAN in data  0
NAN in data  0


In [18]:
train_ts = data_to_np_tensor(train_data)
test_ts = data_to_np_tensor(test_data)

train_ts.shape, test_ts.shape

((28, 132, 5), (28, 24, 5))

In [19]:
model = TS2Vec(
    input_dims=train_ts.shape[2],
    device='cpu', # 2
    output_dims=128
)

loss_log = model.fit(
    train_ts,
    verbose=False
)

In [20]:
train_repr = model.encode(train_ts)
test_repr = model.encode(test_ts)

train_repr.shape, test_repr.shape

((28, 132, 128), (28, 24, 128))

In [21]:
def stock_embeddigns_to_df(data_tensor: np.ndarray, stocks, dates) -> dict:    
    res = pd.DataFrame()
    for i, stock in enumerate(stocks):
        df = pd.DataFrame(data_tensor[i], index=dates)
        df.columns = ['emb_' + str(i) for i in range(len(df.columns))]
        df['Stock'] = stock
        res = pd.concat([res, df])

    return res.reset_index(drop=False, names='Datetime')

X_train = stock_embeddigns_to_df(train_repr, stocks=train_data['Open'].columns, dates=train_data['Open'].index)
X_test = stock_embeddigns_to_df(test_repr, stocks=test_data['Open'].columns, dates=test_data['Open'].index)
X_train.head()

,Datetime,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,...,emb_119,emb_120,emb_121,emb_122,emb_123,emb_124,emb_125,emb_126,emb_127,Stock
0,2023-10-02 17:00:00,-0.235238,-0.070557,0.119213,0.005907,-0.071874,-0.092587,0.182050,-0.070452,-0.504693,...,-0.341755,-0.049969,0.045584,0.032349,-0.136634,-0.077857,-0.328544,0.019047,0.272873,AAPL
1,2023-10-02 18:00:00,-0.251850,-0.041827,-0.203974,-0.325048,0.242328,-0.201659,-0.202775,-0.235551,-0.133075,...,-0.313413,-0.554562,-0.153764,-0.283301,-0.325600,-0.024445,-0.272005,-0.396758,0.264312,AAPL
2,2023-10-02 19:00:00,-0.000953,0.035200,0.179547,-0.035827,-0.231748,-0.087567,0.144272,-0.157403,-0.357360,...,-0.278717,-0.077809,0.090696,-0.049918,-0.157598,-0.030466,-0.133280,-0.213699,0.217249,AAPL
3,2023-10-02 20:00:00,-0.188639,-0.053336,-0.223995,-0.202868,0.076745,-0.285764,-0.039586,-0.057483,-0.286652,...,-0.237493,-0.392561,-0.195927,-0.174810,-0.285698,-0.023573,-0.361939,-0.296779,0.264917,AAPL
4,2023-10-02 21:00:00,-0.269558,-0.107861,0.114273,-0.059645,-0.001949,-0.056198,0.055762,-0.185661,-0.446727,...,-0.441138,-0.366427,0.000013,-0.106111,-0.223219,-0.028051,-0.153794,-0.159955,0.226705,AAPL


In [22]:
X_emd = pd.concat([X_train, X_test])

In [24]:
y = df_best.set_index('Datetime').groupby(
    ['Stock', pd.Grouper( freq='h')],
).agg({'Close': 'mean'}).reset_index()
y.head()

,Stock,Datetime,Close
0,AAPL,2023-01-30 17:00:00,144.833667
1,AAPL,2023-01-30 18:00:00,144.159833
2,AAPL,2023-01-30 19:00:00,143.951083
3,AAPL,2023-01-30 20:00:00,143.681667
4,AAPL,2023-01-30 21:00:00,143.388167


In [25]:
train_start, train_end = '2023-10-01', '2023-11-01'
test_start, test_end = '2023-11-01', '2023-11-07'

y_train = y[(y['Datetime'].dt.date >= pd.Timestamp(train_start).date()) & 
             (y['Datetime'].dt.date < pd.Timestamp(train_end).date())]

y_test = y[(y['Datetime'].dt.date >= pd.Timestamp(test_start).date()) & 
             (y['Datetime'].dt.date < pd.Timestamp(test_end).date())]

In [26]:
y_emd = pd.concat([y_train, y_test])

In [27]:
y_emd

,Stock,Datetime,Close
1011,AAPL,2023-10-02 17:00:00,173.344000
1012,AAPL,2023-10-02 18:00:00,173.639167
1013,AAPL,2023-10-02 19:00:00,173.219500
1014,AAPL,2023-10-02 20:00:00,173.224333
1015,AAPL,2023-10-02 21:00:00,173.018417
...,...,...,...
41851,XOM,2023-11-06 18:00:00,107.347500
41852,XOM,2023-11-06 19:00:00,107.014914
41853,XOM,2023-11-06 20:00:00,106.882000
41854,XOM,2023-11-06 21:00:00,106.277917


In [28]:
# X_emd.to_csv('results/TS2Vec/x_emb.csv', index=False)
# y_emd.to_csv('results/TS2Vec/y.csv', index=False)

In [29]:
y_name = 'Close'

df_train = pd.merge(X_train, y_train, on=['Stock', 'Datetime'])
df_test = pd.merge(X_test, y_test, on=['Stock', 'Datetime'])

X_train, y_train = df_train.drop(columns=[y_name, 'Datetime', 'Stock']), df_train[y_name]
X_test, y_test = df_test.drop(columns=[y_name, 'Datetime', 'Stock']), df_test[y_name]

In [30]:
model = XGBRegressor()

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

MAE(y_test, y_pred)

91.89726627898725

# Baseline

In [31]:
X_baseline = df.set_index('Datetime').groupby(
    ['Stock', pd.Grouper( freq='h')],
).agg({'Close': 'mean'}).reset_index()
X_baseline.head()

,Stock,Datetime,Close
0,AAPL,2023-01-30 17:00:00,NaN
1,AAPL,2023-01-30 18:00:00,-0.001171
2,AAPL,2023-01-30 19:00:00,-0.006967
3,AAPL,2023-01-30 20:00:00,0.001528
4,AAPL,2023-01-30 21:00:00,-0.005480


In [32]:
orig_vals = df_best_h.reset_index()

In [33]:
n_shifts = 18
for i in range(1, n_shifts + 1):
    X_baseline[f'shift_{i}'] = X_baseline.groupby(by=['Stock']).shift(i)['Close']

X_baseline = X_baseline.dropna()
y = X_baseline.loc[:, :'Close'] 

X_baseline = X_baseline.drop(columns='Close')
X_baseline.head()

,Stock,Datetime,shift_1,shift_2,shift_3,shift_4,shift_5,shift_6,shift_7,shift_8,shift_9,shift_10,shift_11,shift_12,shift_13,shift_14,shift_15,shift_16,shift_17,shift_18
19,AAPL,2023-02-02 18:00:00,0.052279,-0.001548,-0.001756,-0.001332,-0.002100,-0.006812,0.001392,0.001464,0.001186,0.000489,0.005476,-0.000140,-0.006001,-0.000418,-0.005480,0.001528,-0.006967,-0.001171
20,AAPL,2023-02-02 19:00:00,-0.003817,0.052279,-0.001548,-0.001756,-0.001332,-0.002100,-0.006812,0.001392,0.001464,0.001186,0.000489,0.005476,-0.000140,-0.006001,-0.000418,-0.005480,0.001528,-0.006967
21,AAPL,2023-02-02 20:00:00,0.008334,-0.003817,0.052279,-0.001548,-0.001756,-0.001332,-0.002100,-0.006812,0.001392,0.001464,0.001186,0.000489,0.005476,-0.000140,-0.006001,-0.000418,-0.005480,0.001528
22,AAPL,2023-02-02 21:00:00,-0.001400,0.008334,-0.003817,0.052279,-0.001548,-0.001756,-0.001332,-0.002100,-0.006812,0.001392,0.001464,0.001186,0.000489,0.005476,-0.000140,-0.006001,-0.000418,-0.005480
23,AAPL,2023-02-02 22:00:00,0.001602,-0.001400,0.008334,-0.003817,0.052279,-0.001548,-0.001756,-0.001332,-0.002100,-0.006812,0.001392,0.001464,0.001186,0.000489,0.005476,-0.000140,-0.006001,-0.000418


In [34]:
train_start, train_end = '2023-10-01', '2023-11-01'
test_start, test_end = '2023-11-01', '2023-11-07'

y_train = y[(y['Datetime'].dt.date >= pd.Timestamp(train_start).date()) & 
             (y['Datetime'].dt.date < pd.Timestamp(train_end).date())]['Close']

y_test = y[(y['Datetime'].dt.date >= pd.Timestamp(test_start).date()) & 
             (y['Datetime'].dt.date < pd.Timestamp(test_end).date())]['Close']

X_train = X_baseline[(X_baseline['Datetime'].dt.date >= pd.Timestamp(train_start).date()) & 
             (X_baseline['Datetime'].dt.date < pd.Timestamp(train_end).date())]

X_test = X_baseline[(X_baseline['Datetime'].dt.date >= pd.Timestamp(test_start).date()) & 
             (X_baseline['Datetime'].dt.date < pd.Timestamp(test_end).date())]

last_train_date = X_train.groupby(['Stock'], as_index=False).last()[['Stock', 'Datetime']]
y_start_test = orig_vals.merge(last_train_date, how='inner', on=['Stock', 'Datetime'])
y_start_test.head()

,Stock,Datetime,Open,High,Low,Close,Volume
0,AAPL,2023-10-31 22:00:00,170.850,170.900,170.385,170.62,338198
1,ABBV,2023-10-31 22:00:00,141.420,141.650,139.910,140.19,86557
2,ABT,2023-10-31 22:00:00,94.530,94.880,94.350,94.84,56242
3,AMD,2023-10-31 22:00:00,98.490,98.800,98.100,98.44,386922
4,BAC,2023-10-31 22:00:00,26.355,26.375,26.275,26.28,579334


In [35]:
X_train.head()

,Stock,Datetime,shift_1,shift_2,shift_3,shift_4,shift_5,shift_6,shift_7,shift_8,shift_9,shift_10,shift_11,shift_12,shift_13,shift_14,shift_15,shift_16,shift_17,shift_18
1011,AAPL,2023-10-02 17:00:00,0.005391,-0.003155,-0.004131,-0.003247,0.000464,0.009961,-0.000293,-0.002571,-0.003232,0.008961,0.007638,-0.007288,0.004191,-0.003118,-0.001586,-0.001818,-0.004495,-0.005226
1012,AAPL,2023-10-02 18:00:00,0.008627,0.005391,-0.003155,-0.004131,-0.003247,0.000464,0.009961,-0.000293,-0.002571,-0.003232,0.008961,0.007638,-0.007288,0.004191,-0.003118,-0.001586,-0.001818,-0.004495
1013,AAPL,2023-10-02 19:00:00,-0.000405,0.008627,0.005391,-0.003155,-0.004131,-0.003247,0.000464,0.009961,-0.000293,-0.002571,-0.003232,0.008961,0.007638,-0.007288,0.004191,-0.003118,-0.001586,-0.001818
1014,AAPL,2023-10-02 20:00:00,0.000463,-0.000405,0.008627,0.005391,-0.003155,-0.004131,-0.003247,0.000464,0.009961,-0.000293,-0.002571,-0.003232,0.008961,0.007638,-0.007288,0.004191,-0.003118,-0.001586
1015,AAPL,2023-10-02 21:00:00,0.000693,0.000463,-0.000405,0.008627,0.005391,-0.003155,-0.004131,-0.003247,0.000464,0.009961,-0.000293,-0.002571,-0.003232,0.008961,0.007638,-0.007288,0.004191,-0.003118


In [36]:
model = XGBRegressor()

model.fit(X_train.drop(['Stock', 'Datetime'], axis=1), y_train)
y_pred = model.predict(X_test.drop(['Stock', 'Datetime'], axis=1))

MAE(y_test, y_pred)

0.004322289646521389

In [37]:
df_preds = X_test[['Stock', 'Datetime']].copy()
df_preds['Preds'] = y_pred + 1
df_preds['Close'] = y_test + 1
df_preds.head()

,Stock,Datetime,Preds,Close
1143,AAPL,2023-11-01 17:00:00,1.002082,1.007209
1144,AAPL,2023-11-01 18:00:00,1.001260,0.997498
1145,AAPL,2023-11-01 19:00:00,1.001664,1.002917
1146,AAPL,2023-11-01 20:00:00,0.997889,1.000291
1147,AAPL,2023-11-01 21:00:00,1.000510,1.002675


In [38]:
starts = y_start_test.sort_values('Stock')['Close'].values

In [39]:
pred_close = df_preds.pivot(columns=['Stock'], index='Datetime', values=['Preds']).cumprod() * starts
orig_close = df_preds.pivot(columns=['Stock'], index='Datetime', values=['Close']).cumprod() * starts

pred_close = pred_close['Preds'].reset_index().melt(id_vars=['Datetime'])
orig_close = orig_close['Close'].reset_index().melt(id_vars=['Datetime'])

In [40]:
pred_close.head()

,Datetime,Stock,value
0,2023-11-01 17:00:00,AAPL,170.975290
1,2023-11-01 18:00:00,AAPL,171.190767
2,2023-11-01 19:00:00,AAPL,171.475560
3,2023-11-01 20:00:00,AAPL,171.113558
4,2023-11-01 21:00:00,AAPL,171.200855


In [41]:
orig_close.head()

,Datetime,Stock,value
0,2023-11-01 17:00:00,AAPL,171.85
1,2023-11-01 18:00:00,AAPL,171.42
2,2023-11-01 19:00:00,AAPL,171.92
3,2023-11-01 20:00:00,AAPL,171.97
4,2023-11-01 21:00:00,AAPL,172.43


## Baseline + TS2Vec

In [ ]:
shifts = X_train

In [ ]:
shifts_test = X_test

In [ ]:
def stock_embeddigns_to_df(data_tensor: np.ndarray, stocks, dates) -> dict:    
    res = pd.DataFrame()
    for i, stock in enumerate(stocks):
        df = pd.DataFrame(data_tensor[i], index=dates)
        df.columns = ['emb_' + str(i) for i in range(len(df.columns))]
        df['Stock'] = stock
        res = pd.concat([res, df])

    return res.reset_index(drop=False, names='Datetime')

X_train = stock_embeddigns_to_df(train_repr, stocks=train_data['Open'].columns, dates=train_data['Open'].index)
X_test = stock_embeddigns_to_df(test_repr, stocks=test_data['Open'].columns, dates=test_data['Open'].index)
X_train.head()

In [ ]:
y = df_best.set_index('Datetime').groupby(
    ['Stock', pd.Grouper( freq='h')],
).agg({'Close': 'mean'}).reset_index()
y.head()

In [ ]:
train_start, train_end = '2023-10-01', '2023-11-01'
test_start, test_end = '2023-11-01', '2023-11-07'

y_train = y[(y['Datetime'].dt.date >= pd.Timestamp(train_start).date()) & 
             (y['Datetime'].dt.date < pd.Timestamp(train_end).date())]

y_test = y[(y['Datetime'].dt.date >= pd.Timestamp(test_start).date()) & 
             (y['Datetime'].dt.date < pd.Timestamp(test_end).date())]

In [ ]:
y_name = 'Close'

df_train = pd.merge(X_train, y_train, on=['Stock', 'Datetime'])
df_test = pd.merge(X_test, y_test, on=['Stock', 'Datetime'])

X_train, y_train = df_train.drop(columns=[y_name, 'Datetime', 'Stock']), df_train[y_name]
X_test, y_test = df_test.drop(columns=[y_name, 'Datetime', 'Stock']), df_test[y_name]

In [ ]:
X_train = pd.concat([X_train, shifts], axis=1)
X_test = pd.concat([X_test, shifts_test], axis=1)

In [ ]:
X_train

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error as MAE
from xgboost import XGBRegressor

model = XGBRegressor()

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

MAE(y_test, y_pred)